<a href="https://colab.research.google.com/github/Tikam18/BestPractices/blob/master/web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Changing the date in data from 1980-2022

In [ ]:
from email.header import Header
import json
import requests
import pandas as pd
import numpy as np
#print("start program")
API_KEY = "8cba6536456c2704bdd5fb1333b65f0e"
query = "supercapacitor"
base_url = 'https://api.elsevier.com/content/search/sciencedirect'

data = {"qs": query,
        "date": 1990,
        #"volume": 0,
        "display": {
            "show": 100,
            "offset": 0
        }}

headers = {'x-els-apikey': API_KEY,
           'Content-Type': 'application/json',
           'Accept': 'application/json',
           'email': 'tcsoni.phd2018.meta@nitrr.ac.in'}

## Getting the response from Science Direct

In [ ]:
def get_response(url, data, headers):
    """
    Return the response from Elsevier
    :param url: <str> base_url
    :param data: <dict> data parameters
    :param headers: <dict> headers
    :return: response
    """
    response = requests.put(url, data=json.dumps(data), headers=headers)
    response = response.text.replace('false', 'False').replace('true', 'True')
    try:
        response = eval(response)
    except BaseException:
        print(response)
    return response

In [ ]:
print(get_response(base_url,data,headers))

## Getting DOI from the response

In [ ]:
def get_doi(data):
    """
    Get DOIs
    :param data: <dict> data parameters
    :param volume: <int> the volume index
    :param year: <int> the year index
    :return: <list> of <str> list of DOIs
    """
    dois = []
    # data['volume'] = volume
    #data["date"] = year
    response = get_response(base_url, data, headers)
    if 'resultsFound' in response.keys():
        n = int(np.ceil(response['resultsFound'] / 100))
    else:
        n = 60
    for offset in range(n + 1):
        data["display"]["offset"] = offset
        response = get_response(base_url, data, headers)
        if 'results' in response.keys():
            results = response['results']
            for result in results:
                if 'doi' in result:
                    dois.append(result['doi'])
    return dois

Returns a list of doi's for particular year.

In [ ]:
dois = get_doi(data)
print(dois)
print(len(dois))

Removing the duplicates


In [ ]:
dois = list( dict.fromkeys(dois) )
print(dois)

In [ ]:
new_dois = []
for item in dois:
    for i in item:
        item=item.replace("/","")
    new_dois.append(item)
print(new_dois)
print(len(new_dois))

## Downloading paper in xml format according to the DOIs

In [ ]:
def download_doi(doi):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    for i in doi:
        ndoi=doi.replace("/","")
    
    with open(str(ndoi) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(doi, API_KEY)
        text = requests.get(request_url).text
        f.write(text)
    return

In [ ]:
for d in dois:
    download_doi(d)